![](img/memento.py.png)

---

In [1]:
"""
http://code.activestate.com/recipes/413838-memento-closure/

*TL;DR
Provides the ability to restore an object to its previous state.
"""

from typing import Callable, List
from copy import copy, deepcopy


def memento(obj, deep=False):
    state = deepcopy(obj.__dict__) if deep else copy(obj.__dict__)

    def restore():
        obj.__dict__.clear()
        obj.__dict__.update(state)

    return restore


class Transaction:
    """A transaction guard.

    This is, in fact, just syntactic sugar around a memento closure.
    """

    deep = False
    states: List[Callable[[], None]] = []

    def __init__(self, deep, *targets):
        self.deep = deep
        self.targets = targets
        self.commit()

    def commit(self):
        self.states = [memento(target, self.deep) for target in self.targets]

    def rollback(self):
        for a_state in self.states:
            a_state()


class Transactional:
    """Adds transactional semantics to methods. Methods decorated  with

    @Transactional will rollback to entry-state upon exceptions.
    """

    def __init__(self, method):
        self.method = method

    def __get__(self, obj, T):
        """
        A decorator that makes a function transactional.

        :param method: The function to be decorated.
        """

        def transaction(*args, **kwargs):
            state = memento(obj)
            try:
                return self.method(obj, *args, **kwargs)
            except Exception as e:
                state()
                raise e

        return transaction


class NumObj:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"<{self.__class__.__name__}: {self.value!r}>"

    def increment(self):
        self.value += 1

    @Transactional
    def do_stuff(self):
        self.value = "1111"  # <- invalid value
        self.increment()  # <- will fail and rollback


def main():
    num_obj = NumObj(-1)
    print(num_obj)
    # <NumObj: -1>

    a_transaction = Transaction(True, num_obj)

    try:
       for i in range(3):
           num_obj.increment()
           print(num_obj)
       a_transaction.commit()
       print('-- committed')
       for i in range(3):
           num_obj.increment()
           print(num_obj)
       num_obj.value += 'x'  # will fail
       print(num_obj)
    except Exception:
       a_transaction.rollback()
       print('-- rolled back')
    # <NumObj: 0>
    # <NumObj: 1>
    # <NumObj: 2>
    # -- committed
    # <NumObj: 3>
    # <NumObj: 4>
    # <NumObj: 5>
    # -- rolled back

    print(num_obj)
    # <NumObj: 2>

    print('-- now doing stuff ...')
    # -- now doing stuff ...

    try:
       num_obj.do_stuff()
    except Exception:
       print('-> doing stuff failed!')
       import sys
       import traceback
       traceback.print_exc(file=sys.stdout)
    # -> doing stuff failed!
    # Traceback (most recent call last):
    # ... (10 lines of stack trace)
    # TypeError: ...str...int...

    print(num_obj)
    # <NumObj: 2>
 

main()

# <NumObj: 0>
# <NumObj: 1>
# <NumObj: 2>
# -- committed
# <NumObj: 3>
# <NumObj: 4>
# <NumObj: 5>
# -- rolled back
# <NumObj: 2>
# -- now doing stuff ...
# -> doing stuff failed!
# Traceback (most recent call last):
#   File "/tmp/ipykernel_7499/2360659610.py", line 124, in main
#     num_obj.do_stuff()
#   File "/tmp/ipykernel_7499/2360659610.py", line 66, in transaction
#     raise e
#   File "/tmp/ipykernel_7499/2360659610.py", line 63, in transaction
#     return self.method(obj, *args, **kwargs)
#   File "/tmp/ipykernel_7499/2360659610.py", line 84, in do_stuff
#     self.increment()  # <- will fail and rollback
#   File "/tmp/ipykernel_7499/2360659610.py", line 79, in increment
#     self.value += 1
# TypeError: can only concatenate str (not "int") to str
# <NumObj: 2>

<NumObj: -1>
<NumObj: 0>
<NumObj: 1>
<NumObj: 2>
-- committed
<NumObj: 3>
<NumObj: 4>
<NumObj: 5>
-- rolled back
<NumObj: 2>
-- now doing stuff ...
-> doing stuff failed!
Traceback (most recent call last):
  File "/tmp/ipykernel_7499/2360659610.py", line 124, in main
    num_obj.do_stuff()
  File "/tmp/ipykernel_7499/2360659610.py", line 66, in transaction
    raise e
  File "/tmp/ipykernel_7499/2360659610.py", line 63, in transaction
    return self.method(obj, *args, **kwargs)
  File "/tmp/ipykernel_7499/2360659610.py", line 84, in do_stuff
    self.increment()  # <- will fail and rollback
  File "/tmp/ipykernel_7499/2360659610.py", line 79, in increment
    self.value += 1
TypeError: can only concatenate str (not "int") to str
<NumObj: 2>
